## DQN Agent 😗

In [2]:
import collections
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


from Sim import *

/home/zlxlekta924/anaconda3/envs/test/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
## Replay buffer

class ReplayBuffer():
    '''
    시행한 step 을 하나의 sample 로 저장한다. Network 학습 시 사용된다. 
    :buffer_limit: 메모리에 저장할 sample size
    '''
    
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)

    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)     #sample 메서드 : buffer 중에 n 개만 뽑음
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
                                                        # state 값, action 값, reward 값, 다음 state 값, done_mask 값
                                                        # done_mask : 종료 상태 - 0 과 진행 상태 -1

        for transition in mini_batch:   
            s, a, r, s_prime, done_mask = transition 

            s_lst.append(s)
            a_lst.append([a]) 
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch. float), torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)

    def size(self):
        return len(self.buffer)

In [4]:
class DQN(nn.Module):
    '''
    Agent 가 Action 을 시행할 때 사용하는 Network, 3개의 convolution layer 와 2개의 Fully Conneted layer 로 구성된다.
    Network 를 통해 나온 Action 을 (100 - epsilon)/100 의 확률로 반환한다.
    
    :epsilon: 탐험비율
    '''
    
    def __init__(self):
        super(QnetCNN, self).__init__()
        
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5),
            nn.ReLU())

        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=3),
            nn.ReLU())
        
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=3),
            nn.ReLU())
    
        #Flatten
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(64, 16)
        self.fc2 = nn.Linear(16, 4)
        
    def forward(self,x):
        x = torch.from_numpy(np.asarray(x)).float()
        x = torch.reshape(x, (-1, 1, 10, 9))
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)# 좌우상하 4개의 값을 반환
        return x

    def sample_action(self, obs, epsilon):
        out = self.forward(obs) 
        coin = random.random() 
        

        if coin < epsilon: 
            return random.randint(0,3)
        else:
            return out.argmax().item() 

In [5]:
def train(q, q_target, memory, optimizer): 
    for i in range(10):                                         # 10번 반복학습. 임의로 설정했다.
        s, a, r, s_prime, done_mask = memory.sample(batch_size) # batch_size 개를 버퍼에서 뽑아 모아 놓은 s,a,r,s_prime,done_mask
        q_out = q(s)                                            # s 값으로 다음 각 action 값들의 value 값 반환
        q_a = q_out.gather(1,a)                                 #선택한 액션값들의 q(s,a) 반환
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)  # 다음 state의 각 q(s,a) 값 반환
        target = r + gamma * max_q_prime * done_mask            # gamma : 다음 step 보상의 감가율
        
        loss = F.smooth_l1_loss(q_a, target)                    # DQN 의 손실함수 계산
        optimizer.zero_grad()                                   
        loss.backward()                                         # loss 에 대한 gradient 계산
        optimizer.step()                                        # 손실값을 바탕으로 DQN 의 파라미터 업데이트

In [6]:
def model_save(model_dict, epi):    
    PATH = './weights/'
    torch.save({
            'model': model_dict,
            'epi': epi,
            }, PATH + 'weights.tar')

## Step 🦿

In [23]:
learning_rate = 0.0001 
gamma         = 0.99
buffer_limit  = 100000
batch_size    = 128

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

def main():
    env = Simulator()
    files = pd.read_csv("./data/factory_order_test.csv")

    q = DQN()
    q_target = DQN()
    optimizer = optim.Adam(q.parameters(), lr = learning_rate) 
    memory = ReplayBuffer()

    # checkpoint
    #PATH = './weights/'
    #checkpoint = torch.load(PATH+'weights.tar')
    #q.load_state_dict(checkpoint['model'])
    #q.eval()
    
    q_target.load_state_dict(q.state_dict())      
    score = 0.0
    cnt = 0

    for epi in tqdm(range(len(files))):
        if epi%1000 == 0:
            model_save(q.state_dict(), epi)
            
        epsilon = 0.0
        masking = False

        ep = epi%2380

        s = env.reset(epi)
        obs = np.asarray(s, dtype=np.float32) 

        a_step = 0
        done = False 
        first = True
        
        while not done: 

            if first:
                a = 0
                first = False
                
            else:
                a = q.sample_action(torch.from_numpy(obs).float(), epsilon) 
            
            s_prime, r, cumul, done, goal_of_reward = env.step(a)
            
            if done:
                cnt += 1
                masking = True
            
            if a_step == 300:
                done = True
            
            array = s_prime
            s_prime = np.asarray(s_prime, dtype=np.float32)
            obs = s_prime 

            done_mask = 0.0 if done else 1.0
            memory.put((obs, a, r/100, s_prime, done_mask))

            score += r
            a_step +=1 

    print(f'총 {len(files)} 개의 test 데이터 중 {cnt} 개 CLEAR : 성공률 {round(cnt/len(files), 2)}')
    print(f'총 {cnt} 개의 CLEAR 데이터 총 Action 수는 {env.ac()} 이며, Episode 당 평균 Action 수는 {round(env.ac()/cnt,2)}')

main()

100%|██████████████████████████████████████████████████████████████████████| 1226/1226 [00:28<00:00, 43.65it/s]

총 1226 개의 test 데이터 중 1211 개 CLEAR : 성공률 0.99
총 1211 개의 CLEAR 데이터 총 Action 수는 48066 이며, Episode 당 평균 Action 수는 39.69


## Episode Example 🐚

<img src="./agent.gif" width=300px>